In [1]:
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split , KFold
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import pandas as pd
import math
from bayes_opt import BayesianOptimization
import random
import sklearn
from sklearn.metrics import mean_squared_error
import tensorflow as tf
cocomo=pd.read_csv("C:\\Users\\Asus\\Desktop\\Tehran university\\Seminar\\Datasets\\cocomo81_dataset.csv",header=None)
columns_cocomo=['rely','data','cplx','time','stor','virt','turn','acap','aexp','pcap','vexp','lexp','modp','tool','sced','loc','actual']
cocomo.set_axis(columns_cocomo,axis='columns',inplace=True)
cocomo.set_axis(range(1,64),axis=0 ,inplace=True)
cocomo.rename_axis("Features", axis=1,inplace=True)
cocomo.rename_axis("Projects", axis=0,inplace=True)

In [2]:


# Step 1: Collect Data (Replace with your actual data)
# Load your dataset into 'X' (features) and 'y' (effort/target variable)
# Feature matrix X should have shape (n_samples, n_features)
X = cocomo.drop('actual',axis=1).values
# Target values y should have shape (n_samples,)
y = cocomo['actual'].values

seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Step 2: Preprocess Data
#scaler = StandardScaler()
#X_scaled = scaler.fit_transform(X)

# Step 3: Feature Selection using Harmony Search
def objective_function(subset):
    selected_features = [feature for feature, is_selected in zip(range(X.shape[1]), subset) if is_selected]
    if len(selected_features) == 0:
        return float('-inf')  # Penalize subsets with no selected features
    
    num_folds = 5
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
    
    X_selected = X[:, selected_features]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_selected)
    error_list=[]
    
    for train_index, val_index in kf.split(X_scaled):
        X_train, X_test = X_scaled[train_index], X_scaled[val_index]
        y_train, y_test = y[train_index], y[val_index]
    
        # Step 3: Build the ANN model.
        model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(units=32, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(units=16, activation='relu'),
        tf.keras.layers.Dense(units=1)  # Output layer with a single unit for regression.
    ])

        # Step 4: Compile the model.
        model.compile(optimizer='adam', loss='mean_squared_error')

        # Step 5: Train the model.
        model.fit(X_train, y_train, epochs=5, batch_size=8, verbose=1)

        # Step 6: Evaluate the model.
        y_pred = model.predict(X_test)
    
        error = np.mean(np.abs(y_pred - y_test))
        error_list.append(error)
    
    return 1 / (1 + np.mean(error_list))  # Fitness is the inverse of the error (higher is better)

#==================================================================================================================
# Harmony Search Parameters
num_features = X.shape[1]
hms = 20  # Harmony memory size
iterations = 100  # Number of iterations

# Initialize harmony memory
harmony_memory = np.random.randint(0, 2, size=(hms, num_features))

# Harmony Search Algorithm
for _ in range(iterations):
    rnd_choice=np.random.choice(hms)
    new_harmony = np.copy(harmony_memory[rnd_choice])
    for i in range(num_features):
        if np.random.rand() < 0.5:  # Adjust this probability based on your problem
            new_harmony[i] = 1 - new_harmony[i]
    current_obj = objective_function(harmony_memory[rnd_choice])
    new_obj = objective_function(new_harmony)
    if new_obj > current_obj:
        harmony_memory[rnd_choice] = new_harmony

# Get selected features based on final harmony memory
selected_features = harmony_memory[np.argmax([objective_function(i) for i in harmony_memory])]
print(selected_features)
# Step 4: Train SVR Model using selected features
#selected_X = X_scaled[:, np.array(selected_features) == 1]
#=============================================================================================================               
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
    
X_selected = X[:, selected_features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)
mean_RMSE=[]
mean_MMRE=[]
mean_MAE=[]
    
for train_index, val_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[val_index]
    y_train, y_test = y[train_index], y[val_index]
    
    # Step 3: Build the ANN model.
    model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=32, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(units=16, activation='relu'),
    tf.keras.layers.Dense(units=1)  # Output layer with a single unit for regression.
])

    # Step 4: Compile the model.
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Step 5: Train the model.
    model.fit(X_train, y_train, epochs=5, batch_size=8, verbose=1)

    # Step 6: Evaluate the model.
    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mean_MAE.append(mae)


    # Calculate Mean Magnitude of Relative Error (MMRE)
    mmre = np.mean(np.abs((y_test - y_pred) / y_test))
    mean_MMRE.append(mmre)

    # Calculate the Root Mean Squared Error (RMSE) to assess the model's performance.
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mean_RMSE.append(rmse)
                
print(f"Mean Absolute Error mean: {np.mean(mean_MAE)}")               
print(f"Mean Magnitude of Relative Error mean (MMRE): {np.mean(mean_MMRE):.2f}")            
print(f"Root Mean Squared Error (RMSE) mean: {np.mean(mean_RMSE)}")
print(selected_features)


Epoch 1/5
7/7 [==============================] - 0s 1ms/step - loss: 4595646.5000
Epoch 2/5
7/7 [==============================] - 0s 917us/step - loss: 4595395.0000
Epoch 3/5
7/7 [==============================] - 0s 834us/step - loss: 4595142.0000
Epoch 4/5
7/7 [==============================] - 0s 919us/step - loss: 4594862.5000
Epoch 5/5
1/1 [==============================] - 0s 51ms/step
Epoch 1/5
7/7 [==============================] - 0s 1000us/step - loss: 1276763.0000
Epoch 2/5
7/7 [==============================] - 0s 847us/step - loss: 1276602.0000
Epoch 3/5
7/7 [==============================] - 0s 1ms/step - loss: 1276418.5000
Epoch 4/5
7/7 [==============================] - 0s 1ms/step - loss: 1276208.8750
Epoch 5/5
1/1 [==============================] - 0s 34ms/step
Epoch 1/5
7/7 [==============================] - 0s 978us/step - loss: 4655580.0000
Epoch 2/5
7/7 [==============================] - 0s 1ms/step - loss: 4655309.0000
Epoch 3/5
7/7 [============================

Epoch 4/5
7/7 [==============================] - 0s 834us/step - loss: 3606675.5000
Epoch 5/5
1/1 [==============================] - 0s 31ms/step
Epoch 1/5
7/7 [==============================] - 0s 1ms/step - loss: 4594478.5000
Epoch 2/5
7/7 [==============================] - 0s 919us/step - loss: 4594263.0000
Epoch 3/5
7/7 [==============================] - 0s 1ms/step - loss: 4594045.5000
Epoch 4/5
7/7 [==============================] - 0s 666us/step - loss: 4593822.5000
Epoch 5/5
1/1 [==============================] - 0s 35ms/step
Epoch 1/5
7/7 [==============================] - 0s 1ms/step - loss: 1277355.0000
Epoch 2/5
7/7 [==============================] - 0s 833us/step - loss: 1277229.6250
Epoch 3/5
7/7 [==============================] - 0s 859us/step - loss: 1277094.0000
Epoch 4/5
7/7 [==============================] - 0s 919us/step - loss: 1276949.2500
Epoch 5/5
1/1 [==============================] - 0s 33ms/step
Epoch 1/5
7/7 [==============================] - 0s 917us/step -

Epoch 1/5
7/7 [==============================] - 0s 1ms/step - loss: 4655409.5000
Epoch 2/5
7/7 [==============================] - 0s 1ms/step - loss: 4655109.0000
Epoch 3/5
7/7 [==============================] - 0s 666us/step - loss: 4654794.5000
Epoch 4/5
7/7 [==============================] - 0s 666us/step - loss: 4654538.5000
Epoch 5/5
1/1 [==============================] - 0s 32ms/step
Epoch 1/5
7/7 [==============================] - 0s 1ms/step - loss: 4511918.5000
Epoch 2/5
7/7 [==============================] - 0s 918us/step - loss: 4511766.0000
Epoch 3/5
7/7 [==============================] - 0s 834us/step - loss: 4511641.0000
Epoch 4/5
7/7 [==============================] - 0s 855us/step - loss: 4511529.0000
Epoch 5/5
1/1 [==============================] - 0s 32ms/step
Epoch 1/5
7/7 [==============================] - 0s 1ms/step - loss: 3608427.2500
Epoch 2/5
7/7 [==============================] - 0s 844us/step - loss: 3608222.7500
Epoch 3/5
7/7 [=============================

7/7 [==============================] - 0s 1ms/step - loss: 3608082.2500
Epoch 4/5
7/7 [==============================] - 0s 833us/step - loss: 3607922.0000
Epoch 5/5
1/1 [==============================] - 0s 34ms/step
Epoch 1/5
7/7 [==============================] - 0s 918us/step - loss: 4595138.0000
Epoch 2/5
7/7 [==============================] - 0s 999us/step - loss: 4594966.0000
Epoch 3/5
7/7 [==============================] - 0s 833us/step - loss: 4594789.5000
Epoch 4/5
7/7 [==============================] - 0s 1ms/step - loss: 4594600.5000
Epoch 5/5
1/1 [==============================] - 0s 34ms/step
Epoch 1/5
7/7 [==============================] - 0s 1ms/step - loss: 1277153.6250
Epoch 2/5
7/7 [==============================] - 0s 1ms/step - loss: 1277027.0000
Epoch 3/5
7/7 [==============================] - 0s 918us/step - loss: 1276895.8750
Epoch 4/5
7/7 [==============================] - 0s 1000us/step - loss: 1276745.3750
Epoch 5/5
1/1 [==============================] - 0s 3

1/1 [==============================] - 0s 32ms/step
Epoch 1/5
7/7 [==============================] - 0s 1ms/step - loss: 4655453.5000
Epoch 2/5
7/7 [==============================] - 0s 1000us/step - loss: 4655076.0000
Epoch 3/5
7/7 [==============================] - 0s 1ms/step - loss: 4654712.0000
Epoch 4/5
7/7 [==============================] - 0s 887us/step - loss: 4654389.0000
Epoch 5/5
1/1 [==============================] - 0s 33ms/step
Epoch 1/5
7/7 [==============================] - 0s 1ms/step - loss: 4510862.5000
Epoch 2/5
7/7 [==============================] - 0s 921us/step - loss: 4510616.0000
Epoch 3/5
7/7 [==============================] - 0s 919us/step - loss: 4510362.5000
Epoch 4/5
7/7 [==============================] - 0s 833us/step - loss: 4510110.0000
Epoch 5/5
1/1 [==============================] - 0s 32ms/step
Epoch 1/5
7/7 [==============================] - 0s 1ms/step - loss: 3608889.0000
Epoch 2/5
7/7 [==============================] - 0s 752us/step - loss: 3608

...

1/1 [==============================] - 0s 34ms/step
Epoch 1/5
7/7 [==============================] - 0s 859us/step - loss: 4511654.0000
Epoch 2/5
7/7 [==============================] - 0s 833us/step - loss: 4511443.0000
Epoch 3/5
7/7 [==============================] - 0s 833us/step - loss: 4511227.5000
Epoch 4/5
7/7 [==============================] - 0s 892us/step - loss: 4511015.5000
Epoch 5/5
1/1 [==============================] - 0s 33ms/step
Epoch 1/5
7/7 [==============================] - 0s 919us/step - loss: 3608702.7500
Epoch 2/5
7/7 [==============================] - 0s 752us/step - loss: 3608470.2500
Epoch 3/5
7/7 [==============================] - 0s 752us/step - loss: 3608316.5000
Epoch 4/5
7/7 [==============================] - 0s 833us/step - loss: 3608148.5000
Epoch 5/5
1/1 [==============================] - 0s 32ms/step
Epoch 1/5
7/7 [==============================] - 0s 833us/step - loss: 4594194.0000
Epoch 2/5
7/7 [==============================] - 0s 833us/step - los

7/7 [==============================] - 0s 919us/step - loss: 4594357.0000
Epoch 2/5
7/7 [==============================] - 0s 752us/step - loss: 4594117.0000
Epoch 3/5
7/7 [==============================] - 0s 834us/step - loss: 4593861.5000
Epoch 4/5
7/7 [==============================] - 0s 1ms/step - loss: 4593587.5000
Epoch 5/5
1/1 [==============================] - 0s 34ms/step
Epoch 1/5
7/7 [==============================] - 0s 1000us/step - loss: 1277221.0000
Epoch 2/5
7/7 [==============================] - 0s 846us/step - loss: 1277058.5000
Epoch 3/5
7/7 [==============================] - 0s 918us/step - loss: 1276886.5000
Epoch 4/5
7/7 [==============================] - 0s 833us/step - loss: 1276713.2500
Epoch 5/5
1/1 [==============================] - 0s 32ms/step
Epoch 1/5
7/7 [==============================] - 0s 1ms/step - loss: 4656319.0000
Epoch 2/5
7/7 [==============================] - 0s 847us/step - loss: 4655979.5000
Epoch 3/5
7/7 [==============================] - 

7/7 [==============================] - 0s 1ms/step - loss: 4653661.0000
Epoch 4/5
7/7 [==============================] - 0s 833us/step - loss: 4653259.0000
Epoch 5/5
1/1 [==============================] - 0s 33ms/step
Epoch 1/5
7/7 [==============================] - 0s 1ms/step - loss: 4511885.0000
Epoch 2/5
7/7 [==============================] - 0s 666us/step - loss: 4511715.5000
Epoch 3/5
7/7 [==============================] - 0s 905us/step - loss: 4511537.5000
Epoch 4/5
7/7 [==============================] - 0s 918us/step - loss: 4511369.0000
Epoch 5/5
1/1 [==============================] - 0s 34ms/step
Epoch 1/5
7/7 [==============================] - 0s 1ms/step - loss: 3608556.0000
Epoch 2/5
7/7 [==============================] - 0s 860us/step - loss: 3608374.0000
Epoch 3/5
7/7 [==============================] - 0s 667us/step - loss: 3608229.7500
Epoch 4/5
7/7 [==============================] - 0s 665us/step - loss: 3608056.7500
Epoch 5/5
1/1 [==============================] - 0s 

1/1 [==============================] - 0s 32ms/step
Epoch 1/5
7/7 [==============================] - 0s 999us/step - loss: 4595492.5000
Epoch 2/5
7/7 [==============================] - 0s 666us/step - loss: 4595342.0000
Epoch 3/5
7/7 [==============================] - 0s 918us/step - loss: 4595194.0000
Epoch 4/5
7/7 [==============================] - 0s 751us/step - loss: 4595040.5000
Epoch 5/5
1/1 [==============================] - 0s 31ms/step
Epoch 1/5
7/7 [==============================] - 0s 1ms/step - loss: 1277405.6250
Epoch 2/5
7/7 [==============================] - 0s 833us/step - loss: 1277314.6250
Epoch 3/5
7/7 [==============================] - 0s 835us/step - loss: 1277227.5000
Epoch 4/5
7/7 [==============================] - 0s 919us/step - loss: 1277141.2500
Epoch 5/5
1/1 [==============================] - 0s 33ms/step
Epoch 1/5
7/7 [==============================] - 0s 1000us/step - loss: 4656080.5000
Epoch 2/5
7/7 [==============================] - 0s 750us/step - loss

7/7 [==============================] - 0s 919us/step - loss: 4656918.0000
Epoch 2/5
7/7 [==============================] - 0s 833us/step - loss: 4656572.5000
Epoch 3/5
7/7 [==============================] - 0s 919us/step - loss: 4656233.0000
Epoch 4/5
7/7 [==============================] - 0s 852us/step - loss: 4655926.0000
Epoch 5/5
1/1 [==============================] - 0s 34ms/step
Epoch 1/5
7/7 [==============================] - 0s 917us/step - loss: 4510446.0000
Epoch 2/5
7/7 [==============================] - 0s 833us/step - loss: 4510118.0000
Epoch 3/5
7/7 [==============================] - 0s 834us/step - loss: 4509769.0000
Epoch 4/5
7/7 [==============================] - 0s 744us/step - loss: 4509403.5000
Epoch 5/5
1/1 [==============================] - 0s 33ms/step
Epoch 1/5
7/7 [==============================] - 0s 996us/step - loss: 3608243.5000
Epoch 2/5
7/7 [==============================] - 0s 833us/step - loss: 3608059.2500
Epoch 3/5
7/7 [==============================]